In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10
#MIN_number_of_attributes = 35
#MAX_number_of_attributes = 35
number_of_attributes = 35

print(tf.__version__)

2.7.0


In [3]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/02_Atributos_Weve.csv"
df_raw = pd.read_csv(path_training)
df_test_raw = pd.read_csv(path_test)
df_raw
target = df_raw.pop('target')
server_port = df_raw.pop('ServerPort')
client_port = df_raw.pop('ClientPort')

target_test = df_test_raw.pop('target')
server_port_test = df_test_raw.pop('ServerPort')
client_port = df_test_raw.pop('ClientPort')

attributes_array = ['60', '59', '82', '46', '17', '84', '79', '10', '90', '15', '24', '92', '91', '83', '48', '37', '22', '88', '47', '45', '34', '32', '36', '38', '33', '31', '80', '61', '62', '64', '89', '63', '29', '87', '81', '75','76','77','78']
#attributes_array = ['10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']                    

In [4]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#classes = list(target.unique())
#classes
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target.unique()
target_test = target_test.apply(lambda x: classes.index(x))


In [5]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [6]:
#for number_of_attributes in range(MIN_number_of_attributes ,MAX_number_of_attributes+1):
# Select a sub-dataframe from the main dataframe with a lower number of attributes
df = df_raw[attributes_array[0:number_of_attributes]]
df_test = df_test_raw[attributes_array[0:number_of_attributes]]

print(df_test)

# Combine the data with possible classes
numeric_feature_names = attributes_array[0:number_of_attributes]
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing

# Converts pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
#numeric_features_test = tf.convert_to_tensor(numeric_features_test) botei na célula abaixo

# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

# Execute Trainning and Testing
model = get_basic_model()
print('Starting training for',number_of_attributes,'attributes')
history = model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE, callbacks=[history])
train_accuracy = history.history['accuracy']



       60  59    82  46  17   84    79  ...  62  64     89  63  29     87    81
0       1   2   398   1  52  398  1460  ...   1   1   5840   1  14   6432   437
1       1   2   399   1  52  399  1460  ...   1   1   5840   1  14   6432   282
2       1   2   398   1  52  398  1460  ...   1   1   5840   1  14   6432   353
3       1   2   399   1  52  399  1460  ...   1   1   5840   1  14   6432   282
4       1   2   399   1  52  399  1460  ...   1   1   5840   1  14   6432   616
...    ..  ..   ...  ..  ..  ...   ...  ...  ..  ..    ...  ..  ..    ...   ...
23796   6  10    97   6  40   21  1460  ...   1   1  16384   1  20  17520  1460
23797   6  10    97   6  40   21  1360  ...   1   1  16384   1  20  17680  1360
23798   1   1  1460  18  40  759  1460  ...   1   1  32821   1  20  33580   391
23799   2   2   935   2  40  449  1460  ...   1   1  33580   1  20  33580   495
23800   4   2  1460  27  40    2  1460  ...   1   1  31845   1  20  33580   489

[23801 rows x 35 columns]
Starting trai

In [24]:
for x, y in target_test.value_counts().items():
  print(f"{classes[x]} ({x}): {y}")

WWW (0): 18559
MAIL (1): 2726
FTP-DATA (7): 1257
FTP-PASV (3): 344
DATABASE (6): 329
SERVICES (9): 220
MULTIMEDIA (8): 150
FTP-CONTROL (2): 100
P2P (5): 94
ATTACK (4): 19
INTERACTIVE (10): 2
GAMES (11): 1


In [25]:
#CETERIS PARIBUS
var_array = []
weights = []
# create variables to store min and max and median value for each attribut
df_minimum, df_maximum  = numeric_features_test.min(), numeric_features_test.max()

for attr_i, attr in enumerate(numeric_features_test.columns):
  print(end=f"Attribute {attr} resulting classes: ")
  # create an evenly spaced array of 100 value between min and max
  aux = np.linspace(df_minimum[attr], df_maximum[attr], 100)
  var_array.append(aux) 
  # print(var_array) 

  df_median = list(numeric_features_test.median().values)
  # print(df_median)

  #copy the array with median 100 times
  an_array = np.array(df_median)
  repetitions = 100
  df_median_array = np.tile(an_array, (repetitions, 1))
  # print('median values array', df_median_array)    

  for j, val in enumerate(var_array[attr_i]):   
    # if j >= 3: break
    # replace attribute value with variation
    df_median_array[j, attr_i] = val
  
  # print('modificado', df_median_array)
  df_median_array = tf.convert_to_tensor(df_median_array)
  labels = model.predict(df_median_array, batch_size=BATCH_SIZE)
  print(f"{labels.argmax(axis=-1)}\n number of class labels: ({len(set(labels.argmax(axis=-1)))})\n")


Attribute 60 resulting classes: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 number of class labels: (1)

Attribute 59 resulting classes: [0 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
 number of class labels: (2)

Attribute 82 resulting classes: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 number of class labels: (1)

Attribute 46 resulting classes: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0

In [9]:
test_loss, test_acc =  model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE)

# Date for report file
current_date_and_time = datetime.date.today()
current_date_and_time_string = str(current_date_and_time)
# Report file content
report = {
    "datetime": current_date_and_time_string,
    "training_dataset_shape":  df.shape,
    "test_dataset_shape": df_test.shape,
    "attributes": attributes_array[0:df.shape[1]],
    "number_of_attributes": number_of_attributes,
    "accuracy_train": train_accuracy[-1],
    "accuracy_test": test_acc
}
# Writes into the file
title = "/content/drive/MyDrive/IC-Skynet/NNreports/%s"%current_date_and_time_string + " With %d Attributes.json"%number_of_attributes
with open(title, "w") as f:
  json.dump(report, f)
with open(title, "r") as f:
  rep = json.load(f)
display(rep)

#print(rep["datetime"])

2381/2381 - 3s - loss: 0.2762 - accuracy: 0.9510 - 3s/epoch - 1ms/step


{'accuracy_test': 0.9509684443473816,
 'accuracy_train': 0.980010449886322,
 'attributes': ['60',
  '59',
  '82',
  '46',
  '17',
  '84',
  '79',
  '10',
  '90',
  '15',
  '24',
  '92',
  '91',
  '83',
  '48',
  '37',
  '22',
  '88',
  '47',
  '45',
  '34',
  '32',
  '36',
  '38',
  '33',
  '31',
  '80',
  '61',
  '62',
  '64',
  '89',
  '63',
  '29',
  '87',
  '81'],
 'datetime': '2022-02-02',
 'number_of_attributes': 35,
 'test_dataset_shape': [23801, 35],
 'training_dataset_shape': [24863, 35]}

In [15]:
# # create variables to store min and max and median value for each attribut
# df_minimum, df_maximum, df_median  = numeric_features_test.min(), numeric_features_test.max(), numeric_features_test.median()
# print(df_median, df_minimum)

# # create an evenly spaced array of 100 value between min and max
# arr = []
# for x in df_test.columns:
#   aux = np.arange(df_minimum[x], df_maximum[x], (df_maximum[x] - df_minimum[x])/100)
#   arr.append(aux) 
#   #print(arr)

In [16]:
# # #layer0_weights = model.layers[0].get_weights()[0] #weight
# # #layer0_biases  = model.layers[0].get_weights()[1] #bias
# layer1_weights = model.layers[1].get_weights()[0] #weight
# # #layer1_biases  = model.layers[1].get_weights()[1] #bias
# # #print('Layer 1 weights:',layer1_weights)
# # # print('\n')

# weight_attribute = []
# for attr_i, weights in enumerate(layer1_weights):
#   print(attr_i,'aaa')
#   print(weights)
#   weight_attribute.append((weights.sum(), numeric_feature_names[attr_i]))

# weight_attribute = sorted(weight_attribute)

# print(weight_attribute)

# attributes = [x for _, x in weight_attribute]

# print(attributes)

In [12]:
#Acervo

#df['15'].hist()
#df.dtypes

#['ClientPort','10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']

#print(history.history)

#train_accuracy = history.history['accuracy']
#print(train_accuracy)

# xc = range(10)
# plt.figure()
# plt.plot(xc, train_accuracy)

#print('\nPesos da primeira camada')
#print(model.layers[0].weights)
#print('\nPesos da segunda camada: intermediária')
##print(model.layers[1].weights)
#print('\nPesos da terceira camada: final')
#print(model.layers[2].weights)